# Create artifact from time series dataframe
Gets a .tsf or .csv with a time serie, convert int to np.dataframe and loads it to weights and biases (W&B)

## Set-up
Initial notebook setup and specific debugging and pre-configured cases selection
### VsCode update patch
Initial notebook setup when using VSCode

In [1]:
import sys
if '--vscode' in sys.argv:
    print("Executing inside vscode")
    import nbs_pipeline.utils.vscode  as vs
    vs.DisplayHandle.update = vs.update_patch

### Debugging variables
- `print_flag`. If `True` it adds debbuging messages in those functions that allows so (eg. `get_enc_embeddings`)
- `reset_kernel`. If `True` it resets the kernel by the end of the execution. Use only in case that memory management is needed.

In [2]:
print_flag = True
reset_kernel=False

In [3]:
import utils.config as cfg
cfg.show_available_configs()

Available datasets: 
0 - monash_australian_electricity_demand_0
1 - monash_solar_4_seconds_0
2 - wikipedia_0
3 - traffic_san_francisco_0
4 - monash_solar_10_minutes_0
5 - etth1_0
6 - stumpy_abp_0
7 - stumpy_toy_0


In [4]:
pre_configured_case = True
case_id = 1
divide_frequency_factor = 5

## Main code


In [5]:
import pandas as pd
import numpy as np
from fastcore.all import *
import wandb
from dvats.load import TSArtifact, infer_or_inject_freq
import pickle
import matplotlib
import matplotlib.pyplot as plt
from tsai.data.external import convert_tsf_to_dataframe
from tsai.utils import stack_pad

### Path and Artiffact configurattions
This notebook gets configuration from `config\base.yaml` and `config\01-dataset_artifact.yaml`

In [6]:
base_path = Path.home()

In [7]:
config = cfg.get_artifact_config_sd2a(print_flag = False)
if pre_configured_case: 
    cfg.force_artifact_config_sd2a(config, case_id, print_flag = print_flag, both = print_flag)

Selecting  monash_solar_4_seconds_0
artifact_name: Monash-Australian_electricity_demand -> solar_4_seconds
normalize_training: False
data_fpath: ~/data/australian_electricity_demand_dataset.tsf -> ~/data/solar_4_seconds_dataset.tsf
test_split: None
resampling_freq: None
joining_train_test: False
missing_values_technique: None
freq: 1h -> 4s
csv_config: {}
data_cols: [0] -> []
start_date: None
missing_values_constant: None
time_col: None
range_testing: None
use_wandb: True
range_training: None
date_format: %Y-%m-%d %H:%M:%S
wandb_artifacts_path: ./data/wandb_artifacts
date_offset: None


### Data Extraction

The data is assumed to come as a dataframe, either as a binarized  picke file or
as a csv file. It can also come as a `.tsf` file

#### Check file content (if wanted)

In [8]:
if print_flag:
    fpath=os.path.expanduser(config.data_fpath)
    print(fpath)
    try: 
        with open(fpath, 'r') as file:
            for _ in range(13):
                line = file.readline()
                print(line, end='')
        data, _, _, _, _ = convert_tsf_to_dataframe(fpath)
        print("Timestamp", data.start_timestamp)
    except Exception as e:
        print("Error while converting file. Maybe not a tsf: ", e)

/home/macu/data/solar_4_seconds_dataset.tsf
# Dataset Information
# This dataset contains a single very long daily time series representing the solar power production recorded per every 4 seconds starting from 01/08/2019.
# It was downloaded from the Australian Energy Market Operator (AEMO) online platform.
# The length of this time series is 7397222.
#
# For more details, please refer to
# Australian Energy Market Operator, 2020. https://aemo.com.au/
#
@relation Solar_Power
@attribute series_name string
@attribute start_timestamp date
@frequency 4_seconds
@missing false
Timestamp 0   2019-08-01 00:00:03
Name: start_timestamp, dtype: datetime64[ns]


#### Extract data

In [9]:
ext = str(config.data_fpath).split('.')[-1]

if ext == 'pickle':
    df = pd.read_pickle(config.data_fpath)
    
elif ext in ['csv','txt']:
    df = pd.read_csv(config.data_fpath, **config.csv_config)
    
elif ext == 'tsf':
    data, _, _, _, _ = convert_tsf_to_dataframe(os.path.expanduser(config.data_fpath))
    config.update({'start_date': data.start_timestamp[0]}, allow_val_change=True)
    date_format = config.date_format
    df = pd.DataFrame(stack_pad(data.series_value).T)
    
else:
    raise Exception('The data file path has an unsupported extension')


In [10]:
if print_flag:
    print(f'File loaded successfully')
    print(df.shape)
    display(df.head())

File loaded successfully
(7397222, 1)


,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


#### Set the time column (if any) as index

In [11]:
if config.time_col is not None:
    if print_flag: print("time_col: "+str(config.time_col))
    
    if isinstance(config.time_col, int): 
        if print_flag: print("Op 1: time_col int")
        datetime = df.iloc[:, config.time_col]
    
    elif isinstance(config.time_col, list): 
        if print_flag: print("Op 2: time_col list")
        datetime = df.iloc[:, config.time_col].apply(lambda x: x.astype(str).str.cat(sep='-'), axis=1)
    
    index = pd.DatetimeIndex(datetime)
    
    if config.date_offset:
        index += config.date_offset
    
    df = df.set_index(index, drop=False)   
    
    #Delete Timestamp col
    col_name = df.columns[config.time_col]
    
    if print_flag: print("... drop Timestamp col " + str(col_name))
    
    df = df.drop(col_name, axis=1)
    
if print_flag: display(df.head())

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


#### Set dataframe frequency

In [12]:
df = infer_or_inject_freq(
    df, 
    injected_freq=config.freq, 
    start_date=config.start_date, 
    format=config.date_format
)
if print_flag: print(df.index.freq)

<4 * Seconds>


#### Select only the needed variables

In [13]:
# Subset of variables
if config.data_cols:
    if print_flag: print("data_cols: ", config.data_cols)
    df = df.iloc[:, config.data_cols]

if print_flag: print(f'Num. variables: {len(df.columns)}')

Num. variables: 1


#### Select only the needed rows

In [14]:
if divide_frequency_factor > 1: 
    df = df.iloc[::divide_frequency_factor]
if (print_flag): display(df.head())

,0
2019-08-01 00:00:03,0.0
2019-08-01 00:00:23,0.0
2019-08-01 00:00:43,0.0
2019-08-01 00:01:03,0.0
2019-08-01 00:01:23,0.0


In [15]:
if print_flag: print(df.shape)

(1479445, 1)


In [16]:
if print_flag: print(df.index.freq)

<20 * Seconds>


In [19]:
if (divide_frequency_factor > 1): 
    td = pd.to_timedelta(df.index.freq)

    print(td.days)
    print(td.seconds)
    filename = config.data_fpath.split(".tsf", 1)[0]
    sufix = "-"
    if td.days > 0 and td.seconds == pd.to_timedelta(0,'s'):
        df.index.freq = td.days
        sufix =str(td.days)+'d'
    elif td.seconds % 3600 == 0:
        hours = (td.seconds // 3600) + td.days*24
        df.index.freq = pd.to_timedelta(hours, 'h')
        sufix = str(hours)+'h'
    elif td.seconds % 60 == 0: 
        minutes = (td.seconds // 60) + (td.days*24*60)
        df.index.freq = pd.to_timedelta(minutes,'m')
        sufix = str(minutes)+'m'
    else: 
        seconds = td.seconds + td.days*24*60*60
        print(seconds)
        df.index.freq = pd.to_timedelta(seconds, "s")
        sufix = str(seconds)+'s'

    config.artifact_name = config.artifact_name+"-"+sufix
    config.data_fpath = filename+"-"+sufix+".tsf"
    config.freq = sufix
    print(df.index.freq)
    print(config.artifact_name)
    print(config.data_fpath)

0
20
20
<20 * Seconds>
solar_4_seconds-20s
~/data/solar_4_seconds_dataset-20s.tsf


In [ ]:
print(config.freq)
print(config.artifact_name)
print(config.data_fpath)

#### Ensure data integrity

In [ ]:
#Duplicated rows
if print_flag: print("df shape before dropping duplicates", df.shape)
df.drop_duplicates()
if print_flag: print("df shape after dropping duplicates", df.shape)
# Verificar si hay duplicados en el índice del dataframe
if df.index.duplicated().any():
    raise ValueError("Duplicated index names")

In [ ]:
# Replace the default missing values by np.NaN
if config.missing_values_constant:
    df.replace(config.missing_values_constant, np.nan, inplace=True)

#### Show time series plot

In [ ]:
show_time_serie_flag = False
if show_time_serie_flag:
    # Show time series plot
    fig, ax = plt.subplots(1, figsize=(15,5), )
    cmap = matplotlib.colormaps.get_cmap('viridis')
    #df.plot(color=cmap(0.05), ax=ax) # or use colormap=cmap
    df.plot(colormap=cmap, ax=ax) # or use colormap=cmap
    # rect = Rectangle((5000, -4.2), 3000, 8.4, facecolor='lightgrey', alpha=0.5)
    # ax.add_patch(rect)
    plt.tight_layout()
    plt.legend()
    display(plt.show())

### Data Transformation

__Handle Missing Values, Resample and Normalize__

> In this second part, Time Series Artifact (TSArtifact) object can be created and missing values handling techniques, resampling and normalization can be applied.
> 
> This techniques should be applied on the three subsets that must be previously created: training, validation and testing.

#### Training data

##### Build dataframe

In [ ]:
rg = config.range_training

if isinstance(rg, list):
    rg_training = rg
    
elif isinstance(rg, dict):
    rg_training = pd.date_range(rg['start'], rg['end'], freq=rg['freq'])
    
elif config.test_split:
    rg_training = df.index[:math.ceil(len(df) * (1-config.test_split))]

else:
    rg_training = None
    
df_training = df[df.index.isin(rg_training)] if rg_training is not None else df

In [ ]:
df_training.shape

##### Build training artifact

In [ ]:
config.artifact_name

In [ ]:
training_artifact = TSArtifact.from_df(
    df_training, 
    name=config.artifact_name, 
    missing_values_technique=config.missing_values_technique,
    resampling_freq=config.resampling_freq, 
    normalize=config.normalize_training, 
    path=str(Path.home()/config.wandb_artifacts_path)
)
if print_flag: display(training_artifact.metadata)

In [ ]:
#Debugging 
if df_training.index.duplicated().any():
    raise ValueError("Duplicated index names")

#### Testing data

##### Build dataframe & artifact

In [ ]:
# Testing data
rg = config.range_testing

if rg or config.test_split:
    
    if isinstance(rg, list):
        rg_testing = rg

    elif isinstance(rg, dict):
        rg_testing = pd.date_range(rg['start'], rg['end'], freq=rg['freq'])

    elif config.test_split:
        rg_testing = df.index[math.ceil(len(df) * (1 - config.test_split)):]

    else:
        rg_testing = None
    
    df_testing = df[df.index.isin(rg_testing)]
    testing_artifact = TSArtifact.from_df(df_testing,
                                          name=config.artifact_name, 
                                          missing_values_technique=config.missing_values_technique,
                                          resampling_freq=config.resampling_freq, 
                                          normalize=False,
                                          path=str(Path.home()/config.wandb_artifacts_path))
    display(testing_artifact.metadata)
    if df_testing.index.duplicated().any():
        print("Hay valores duplicados en el índice del dataframe.")
    else:
        print("No hay valores duplicados en el índice del dataframe.")
else:
    print("rg "+ str(rg) + " | test_split "+ str(config.test_split))
    testing_artifact = None

#### Training + Testing data

##### Build dataframe & artifact

In [ ]:
# Training + Testing data
if(config.joining_train_test):
    print("joining_train_test: "+ str(config.joining_train_test))
    df_train_test = pd.concat([df_training, df_testing])
    train_test_artifact = TSArtifact.from_df(df_train_test,
                                           name=config.artifact_name, 
                                           missing_values_technique=config.missing_values_technique,
                                           resampling_freq=config.resampling_freq, 
                                           normalize=False,
                                           path=str(Path.home()/config.wandb_artifacts_path))
    if df_train_test.index.duplicated().any():
        print("Hay valores duplicados en el índice del dataframe.")
    else:
        print("No hay valores duplicados en el índice del dataframe.")
    display(train_test_artifact.metadata)
else:
    train_test_artifact = None

### Storing artifacts

For the experiment tracking and hyperparameter we will use the tool **Weights & Biases**. 

> 
Before running this notebook part, make sure you have the `$WANDB_API_KEY`, `$WANDB_ENTITY` and `$WANDB_PROJECT` environment varibales defined with your API_KEY and your ENTITY and PROJECT names (run in a terminal `echo $WANDB_API_KEY` to see it, same with the other variables). If not, run in a terminal `wandb login [API_KEY]` to set the first one. You can see your API_KEY [here](https://wandb.ai/authorize) or in the settings of your W&B account. Run in a terminal `export WANDB_ENTITY=entity_name` and/or `export WANDB_PROJECT=project_name` to set the other two
> 
> <span style="color:red"> TODO: Modify config.ipynb so it gets wandb config from base.yml </span>.

In [ ]:
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="01_dataset_artifact"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
print("runname: "+runname)

In [ ]:
mode = 'online' if config.use_wandb else 'disabled'

# Make the run that will produce the artifact
with wandb.init(job_type='create_dataset', resume=True, mode=mode, config=config, name=runname) as run:
    if testing_artifact: 
        run.log_artifact(training_artifact, aliases=['train'])
        run.log_artifact(testing_artifact, aliases=['test'])
        
        if train_test_artifact:
            run.log_artifact(train_test_artifact, aliases=['all'])
    
    else:
        run.log_artifact(training_artifact, aliases=['all'])

In [ ]:
run.finish()

In [ ]:
from dvats.imports import beep
print("Execution ended")
beep(1)

In [ ]:
if reset_kernel:
    import os
    os._exit(00)